In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
import scipy
from scipy.optimize import curve_fit
import seaborn as sns
from matplotlib import cm
import csv
import scipy.io as sio
import pandas as pd
from scipy.special import erf
import time
import torch
import os
import h5py

# plotting configuration
ratio = 1.7
figure_len, figure_width = 24*ratio, 18*ratio
title_font_size = 12*ratio
font_size_1, font_size_2 = 9*ratio, 9*ratio
legend_size = 18*ratio
line_width, tick_len = ratio, 4*ratio
marker_size = 5*ratio
plot_line_width = 2*ratio
hfont = {'fontname': 'Arial'}
marker_edge_width = 4
pal = sns.color_palette("deep")
sns.set(style='ticks')

In [3]:
N = 30
dx2deg2 = 6 * 6 # (degrees per lattice spacing) squared
tau_E, tau_P, tau_S, tau_V = 0.02, 0.01, 0.01, 0.01 # time constants

def calc_distance(i, j):
    """calculate the distance between two neurons,
        i and j are the indices of two neurons, i and j are from 0 to N*N-1 (31*30),
        x1 and y1 are the coordinate for the first neuron, x1 and y1 are from 0 to 30.
    """
    x1, y1 = divmod(i, N)
    x2, y2 = divmod(j, N)

    xd = np.minimum(np.abs(x1 - x2), N - np.abs(x1 - x2))
    yd = np.minimum(np.abs(y1 - y2), N - np.abs(y1 - y2))
    return np.sqrt(xd**2 + yd**2)

def calc_distance_matrix(N):
    x, y = np.arange(N), np.arange(N)
    x_grid, y_grid = np.meshgrid(x, y, indexing='ij')

    x1, x2 = np.reshape(x_grid, (-1, 1)), np.reshape(x_grid, (1, -1))
    y1, y2 = np.reshape(y_grid, (-1, 1)), np.reshape(y_grid, (1, -1))

    xd = np.minimum(np.abs(x1 - x2), N - np.abs(x1 - x2))
    yd = np.minimum(np.abs(y1 - y2), N - np.abs(y1 - y2))
    return np.sqrt(xd**2 + yd**2)

class NetConnectivity:
    """ 1. connection strengths,
        2. width of the Gaussian connectivity for recurrent connections,
        3. biases.
    """
    def __init__(self):
        # peak of the Gaussian connectivity
        self.EE, self.EP, self.ES, self.EV = 0, 0, 0, 0
        self.PE, self.PP, self.PS, self.PV = 0, 0, 0, 0
        self.SE, self.SP, self.SS, self.SV = 0, 0, 0, 0
        self.VE, self.VP, self.VS, self.VV = 0, 0, 0, 0
        
        self.sigma2EE2, self.sigma2EP2, self.sigma2ES2, self.sigma2EV2 = 2*7**2, 2*5**2, 2*7**2, 2*5**2
        self.sigma2PE2, self.sigma2PP2, self.sigma2PS2, self.sigma2PV2 = 2*5**2, 2*4**2, 2*5**2, 2*4**2
        self.sigma2SE2, self.sigma2SP2, self.sigma2SS2, self.sigma2SV2 = 2*7**2, 2*5**2, 2*7**2, 2*5**2
        self.sigma2VE2, self.sigma2VP2, self.sigma2VS2, self.sigma2VV2 = 2*5**2, 2*4**2, 2*7**2, 2*4**2

class ConnectivityMat:
    def __init__(self):
        self.EEm, self.EPm, self.ESm, self.EVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.PEm, self.PPm, self.PSm, self.PVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.SEm, self.SPm, self.SSm, self.SVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.VEm, self.VPm, self.VSm, self.VVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))

def load_optimized_networks_parameters(conn_file_path):
    data = torch.load(conn_file_path, map_location=torch.device('cpu'))
    optimized_network_tensor = data['gW']
    optimized_network = optimized_network_tensor.detach().cpu().numpy()
    optimized_network = optimized_network[:4, :]
    return optimized_network

def load_conn_mat(J, J_mat, opti_conn_mat, n_dimension):
    bs = np.array([0.5996, 1.35, 0.6818, 0.8437, 0.6092, 0.5996, 0.5996, 0.5996])
    opti_conn_mat[0, :] = opti_conn_mat[0, :]/(2 * np.sqrt(bs[0]))
    opti_conn_mat[1, :] = opti_conn_mat[1, :]/(2 * np.sqrt(bs[1]))
    opti_conn_mat[2, :] = opti_conn_mat[2, :]/(2 * np.sqrt(bs[2]))
    opti_conn_mat[3, :] = opti_conn_mat[3, :]/(2 * np.sqrt(bs[3]))

    J.EE, J.EP, J.ES, J.EV = opti_conn_mat[0, 0], -opti_conn_mat[0, 1], -opti_conn_mat[0, 2], -opti_conn_mat[0, 3]
    J.PE, J.PP, J.PS, J.PV = opti_conn_mat[1, 0], -opti_conn_mat[1, 1], -opti_conn_mat[1, 2], -opti_conn_mat[1, 3]
    J.SE, J.SP, J.SS, J.SV = opti_conn_mat[2, 0], -opti_conn_mat[2, 1], -opti_conn_mat[2, 2], -opti_conn_mat[2, 3]
    J.VE, J.VP, J.VS, J.VV = opti_conn_mat[3, 0], -opti_conn_mat[3, 1], -opti_conn_mat[3, 2], -opti_conn_mat[3, 3]
    
    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    J_mat.EEm = J.EE * np.exp(-d2_matrix / J.sigma2EE2) / (np.pi * J.sigma2EE2)
    J_mat.EPm = J.EP * np.exp(-d2_matrix / J.sigma2EP2) / (np.pi * J.sigma2EP2)
    J_mat.ESm = J.ES * np.exp(-d2_matrix / J.sigma2ES2) / (np.pi * J.sigma2ES2)
    J_mat.EVm = J.EV * np.exp(-d2_matrix / J.sigma2EV2) / (np.pi * J.sigma2EV2)

    J_mat.PEm = J.PE * np.exp(-d2_matrix / J.sigma2PE2) / (np.pi * J.sigma2PE2)
    J_mat.PPm = J.PP * np.exp(-d2_matrix / J.sigma2PP2) / (np.pi * J.sigma2PP2)
    J_mat.PSm = J.PS * np.exp(-d2_matrix / J.sigma2PS2) / (np.pi * J.sigma2PS2)
    J_mat.PVm = J.PV * np.exp(-d2_matrix / J.sigma2PV2) / (np.pi * J.sigma2PV2)

    J_mat.SEm = J.SE * np.exp(-d2_matrix / J.sigma2SE2) / (np.pi * J.sigma2SE2)
    J_mat.SPm = J.SP * np.exp(-d2_matrix / J.sigma2SP2) / (np.pi * J.sigma2SP2)
    J_mat.SSm = J.SS * np.exp(-d2_matrix / J.sigma2SS2) / (np.pi * J.sigma2SS2)
    J_mat.SVm = J.SV * np.exp(-d2_matrix / J.sigma2SV2) / (np.pi * J.sigma2SV2)

    J_mat.VEm = J.VE * np.exp(-d2_matrix / J.sigma2VE2) / (np.pi * J.sigma2VE2)
    J_mat.VPm = J.VP * np.exp(-d2_matrix / J.sigma2VP2) / (np.pi * J.sigma2VP2)
    J_mat.VSm = J.VS * np.exp(-d2_matrix / J.sigma2VS2) / (np.pi * J.sigma2VS2)
    J_mat.VVm = J.VV * np.exp(-d2_matrix / J.sigma2VV2) / (np.pi * J.sigma2VV2)
    
    J_mat_model = np.zeros((n_dimension, n_dimension)) * np.nan
    J_mat_model[:900, :900] = J_mat.EEm
    J_mat_model[:900, 900:1800] = J_mat.EPm
    J_mat_model[:900, 1800:2700] = J_mat.ESm
    J_mat_model[:900, 2700:] = J_mat.EVm
    
    J_mat_model[900:1800, :900] = J_mat.PEm
    J_mat_model[900:1800, 900:1800] = J_mat.PPm
    J_mat_model[900:1800, 1800:2700] = J_mat.PSm
    J_mat_model[900:1800, 2700:] = J_mat.PVm
    
    J_mat_model[1800:2700, :900] = J_mat.SEm
    J_mat_model[1800:2700, 900:1800] = J_mat.SPm
    J_mat_model[1800:2700, 1800:2700] = J_mat.SSm
    J_mat_model[1800:2700, 2700:] = J_mat.SVm
    
    J_mat_model[2700:, :900] = J_mat.VEm
    J_mat_model[2700:, 900:1800] = J_mat.VPm
    J_mat_model[2700:, 1800:2700] = J_mat.VSm
    J_mat_model[2700:, 2700:] = J_mat.VVm
    J_mat_model = dx2deg2 * J_mat_model
    return J_mat_model

### calculate the number of unstable modes of the Jacobian for different subnetworks and the maximal eigenvalue

In [40]:
J = NetConnectivity()
J_mat = ConnectivityMat()

l_baseline = sio.loadmat("data/baselines.mat")['baseline']

l_stimulus = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85]
n_stimulus = len(l_stimulus)

config = 'final_gaussian_classical'
l_selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
l_s_manipulation = ['original', 'Inh_frozen', 'PV_frozen', 'SST_frozen', 'VIP_frozen']
l_title = [s.replace('_', ' ') for s in l_s_manipulation]

n_dimension = 3600

max_eig_mat = np.zeros((len(l_s_manipulation), len(l_selected_indices), n_stimulus))
n_positive_eig_Jac_mat = np.zeros((len(l_s_manipulation), len(l_selected_indices), n_stimulus))

for s_manipulation_idx in range(len(l_s_manipulation)):
    s_manipulation = l_s_manipulation[s_manipulation_idx]
    print(s_manipulation)

    folder_path = 'models/optimized_files_' + config + '/'
    model_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f != '.DS_Store']
    sorted_files = sorted(model_names, key=lambda name: float(name.replace('.pt', '')))
    sorted_numbers = sorted([float(f.replace('.pt', '')) for f in sorted_files])

    for model_name_idx in l_selected_indices:

        j = model_name_idx
        model_name = sorted_files[model_name_idx]
        print(model_name)

        conn_file_path = folder_path + model_name
        optimized_network = load_optimized_networks_parameters(conn_file_path)

        J_mat_model = load_conn_mat(J, J_mat, optimized_network, n_dimension)  # initialize connectivity and bias

        fr_E = sio.loadmat('simulation_data/fr_E_' + config + '_' + model_name + '.mat')['E']
        fr_P = sio.loadmat('simulation_data/fr_P_' + config + '_' + model_name + '.mat')['P']
        fr_S = sio.loadmat('simulation_data/fr_S_' + config + '_' + model_name + '.mat')['S']
        fr_V = sio.loadmat('simulation_data/fr_V_' + config + '_' + model_name + '.mat')['V']

        tau_values = np.zeros(n_dimension)
        tau_values[:900] = 1/tau_E
        tau_values[900:1800] = 1/tau_P
        tau_values[1800:2700] = 1/tau_S
        tau_values[2700:] = 1/tau_V
        tau_mat = np.diag(tau_values) # gain matrix

        for k in range(n_stimulus):
            if k == 0:
                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(l_baseline[0, 0])
                diag_values[900:1800] = 2 * np.sqrt(l_baseline[0, 1])
                diag_values[1800:2700] = 2 * np.sqrt(l_baseline[0, 2])
                diag_values[2700:] = 2 * np.sqrt(l_baseline[0, 3])
                G = np.diag(diag_values) # gain matrix
            else:
                fr_E_sim = fr_E[:, k]
                fr_P_sim = fr_P[:, k]
                fr_S_sim = fr_S[:, k]
                fr_V_sim = fr_V[:, k]

                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(fr_E_sim)
                diag_values[900:1800] = 2 * np.sqrt(fr_P_sim)
                diag_values[1800:2700] = 2 * np.sqrt(fr_S_sim)
                diag_values[2700:] = 2 * np.sqrt(fr_V_sim)
                G = np.diag(diag_values) # gain matrix

            I = np.eye(n_dimension)
            effective_conn_mat = np.matmul(tau_mat, (- I + np.matmul(G, J_mat_model))) #  - I + G * J_mat_model # tau_mat * (- I + G * J_mat_model)

            # construct the effective connectivity matrix
            if s_manipulation == 'Inh_frozen':
                remove = np.arange(900, 3600)
                keep = np.setdiff1d(np.arange(effective_conn_mat.shape[0]), remove) # Keep all indices except the ones in 'remove'
                effective_conn_mat = effective_conn_mat[np.ix_(keep, keep)] # Slice rows and columns
            elif s_manipulation == 'PV_frozen':
                remove = np.arange(900, 1800)
                keep = np.setdiff1d(np.arange(effective_conn_mat.shape[0]), remove) # Keep all indices except the ones in 'remove'
                effective_conn_mat = effective_conn_mat[np.ix_(keep, keep)] # Slice rows and columns
            elif s_manipulation == 'SST_frozen':
                remove = np.arange(1800, 2700)
                keep = np.setdiff1d(np.arange(effective_conn_mat.shape[0]), remove)
                effective_conn_mat = effective_conn_mat[np.ix_(keep, keep)]   
            elif s_manipulation == 'VIP_frozen':
                remove = np.arange(2700, 3600)
                keep = np.setdiff1d(np.arange(effective_conn_mat.shape[0]), remove)
                effective_conn_mat = effective_conn_mat[np.ix_(keep, keep)]          
            else:
                pass

            evals = np.linalg.eigvals(effective_conn_mat)
            
            if k == 6:
                print("stimulus idx: " + str(k))
                print(np.sum(np.array(evals) > 0))
                
            n_positive_eig_Jac_mat[s_manipulation_idx, model_name_idx, k] = np.sum(np.real(np.array(evals)) > 0)
            max_eig_mat[s_manipulation_idx, model_name_idx, k] = np.max(evals.real)

sio.savemat('data/max_eig.mat', mdict={'max_eig': max_eig_mat})
sio.savemat('data/n_positive_real_eig_Jac_mat.mat', mdict={'n_positive_real_eig': n_positive_eig_Jac_mat})

original
2.9814842343e-01.pt
stimulus idx: 6
0
3.0061629415e-01.pt
stimulus idx: 6
0
3.0643698573e-01.pt
stimulus idx: 6
0
3.0751633644e-01.pt
stimulus idx: 6
0
3.0905631185e-01.pt
stimulus idx: 6
0
3.1116414070e-01.pt
stimulus idx: 6
0
3.1124791503e-01.pt
stimulus idx: 6
0
3.1153696775e-01.pt
stimulus idx: 6
0
3.1269198656e-01.pt
stimulus idx: 6
0
3.1403386593e-01.pt
stimulus idx: 6
0
Inh_frozen
2.9814842343e-01.pt
stimulus idx: 6
89
3.0061629415e-01.pt
stimulus idx: 6
137
3.0643698573e-01.pt
stimulus idx: 6
113
3.0751633644e-01.pt
stimulus idx: 6
117
3.0905631185e-01.pt
stimulus idx: 6
121
3.1116414070e-01.pt
stimulus idx: 6
84
3.1124791503e-01.pt
stimulus idx: 6
97
3.1153696775e-01.pt
stimulus idx: 6
129
3.1269198656e-01.pt
stimulus idx: 6
97
3.1403386593e-01.pt
stimulus idx: 6
69
PV_frozen
2.9814842343e-01.pt
stimulus idx: 6
56
3.0061629415e-01.pt
stimulus idx: 6
89
3.0643698573e-01.pt
stimulus idx: 6
98
3.0751633644e-01.pt
stimulus idx: 6
89
3.0905631185e-01.pt
stimulus idx: 6
109

### calculate the smallest eigenvalue and the corresponding eigenvector of the corresponding part of the response matrix

In [11]:
J = NetConnectivity()
J_mat = ConnectivityMat()

l_baseline = sio.loadmat("data/baselines.mat")['baseline']

l_stimulus = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85]
n_stimulus = len(l_stimulus)

config = 'final_gaussian_classical'
l_selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
l_s_manipulation = ['SST_part'] # ['Inh_part', 'PV_part', 'SST_part', 'VIP_part']
l_title = [s.replace('_', ' ') for s in l_s_manipulation]

n_dimension = 3600

for s_manipulation_idx in range(len(l_s_manipulation)):
    s_manipulation = l_s_manipulation[s_manipulation_idx]
    print(s_manipulation)
    
    if s_manipulation_idx == 0:
        min_eig_val_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus))
        min_eig_vec_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus, 2700))
        sec_min_eig_val_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus))
    else:
        min_eig_val_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus))
        min_eig_vec_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus, 900))
        sec_min_eig_val_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus))

    folder_path = 'models/optimized_files_final_gaussian_classical/'
    model_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f != '.DS_Store']
    sorted_files = sorted(model_names, key=lambda name: float(name.replace('.pt', '')))
    sorted_numbers = sorted([float(f.replace('.pt', '')) for f in sorted_files])

    for model_name_idx in l_selected_indices:

        model_name = sorted_files[model_name_idx]
        print(model_name)

        conn_file_path = folder_path + model_name
        optimized_network = load_optimized_networks_parameters(conn_file_path)

        J_mat_model = load_conn_mat(J, J_mat, optimized_network, n_dimension)  # initialize connectivity and bias

        fr_E = sio.loadmat('simulation_data/fr_E_' + config + '_' + model_name + '.mat')['E']
        fr_P = sio.loadmat('simulation_data/fr_P_' + config + '_' + model_name + '.mat')['P']
        fr_S = sio.loadmat('simulation_data/fr_S_' + config + '_' + model_name + '.mat')['S']
        fr_V = sio.loadmat('simulation_data/fr_V_' + config + '_' + model_name + '.mat')['V']

        tau_values = np.zeros(n_dimension)
        tau_values[:900] = 1/tau_E
        tau_values[900:1800] = 1/tau_P
        tau_values[1800:2700] = 1/tau_S
        tau_values[2700:] = 1/tau_V
        tau_mat = np.diag(tau_values) # gain matrix

        for k in range(n_stimulus):
            if k == 0:
                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(l_baseline[0, 0])
                diag_values[900:1800] = 2 * np.sqrt(l_baseline[0, 1])
                diag_values[1800:2700] = 2 * np.sqrt(l_baseline[0, 2])
                diag_values[2700:] = 2 * np.sqrt(l_baseline[0, 3])
                G = np.diag(diag_values) # gain matrix
            else:
                fr_E_sim = fr_E[:, k]
                fr_P_sim = fr_P[:, k]
                fr_S_sim = fr_S[:, k]
                fr_V_sim = fr_V[:, k]

                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(fr_E_sim)
                diag_values[900:1800] = 2 * np.sqrt(fr_P_sim)
                diag_values[1800:2700] = 2 * np.sqrt(fr_S_sim)
                diag_values[2700:] = 2 * np.sqrt(fr_V_sim)
                G = np.diag(diag_values) # gain matrix

            I = np.eye(n_dimension)
            Jac_mat = np.matmul(tau_mat, (- I + np.matmul(G, J_mat_model))) #  - I + G * J_mat_model # tau_mat * (- I + G * J_mat_model)

            rsp_mat = np.matmul(np.matmul(np.linalg.inv(-Jac_mat), tau_mat), G)
            
            # construct the effective connectivity matrix
            if s_manipulation == 'Inh_part':
                rsp_mat_sub = rsp_mat[900:, 900:]
            elif s_manipulation == 'PV_part':
                rsp_mat_sub = rsp_mat[900:1800, 900:1800]
            elif s_manipulation == 'SST_part':
                rsp_mat_sub = rsp_mat[1800:2700, 1800:2700]
            elif s_manipulation == 'VIP_part':
                rsp_mat_sub = rsp_mat[2700:, 2700:]         
            else:
                pass
            
            evals, evecs = np.linalg.eig(rsp_mat_sub)
            if k == 6:
                print("stimulus idx: " + str(k))
                print(np.sum(np.array(evals) < 0))

            # Find index of the minimal eigenvalue
            leading_idx = np.argmin(evals.real)
            
            # Store leading eigenvalue (real part)
            min_eig_val_rsp_mat[model_name_idx, k] = evals[leading_idx]
            min_eig_vec_rsp_mat[model_name_idx, k, :] = evecs[:, leading_idx]
            
    sio.savemat('data/min_eig_val_rsp_' + s_manipulation + '.mat', mdict={'min_eig_val': min_eig_val_rsp_mat})
    sio.savemat('data/min_eig_vec_rsp_' + s_manipulation + '.mat', mdict={'min_eig_vec': min_eig_vec_rsp_mat})

SST_part
2.9814842343e-01.pt
stimulus idx: 0
0
stimulus idx: 1
0
stimulus idx: 2
0
stimulus idx: 3
0
stimulus idx: 4
0
stimulus idx: 5
1
stimulus idx: 6
3
stimulus idx: 7
3
stimulus idx: 8
3
stimulus idx: 9
3
3.0061629415e-01.pt
stimulus idx: 0
0
stimulus idx: 1
0
stimulus idx: 2
0
stimulus idx: 3
0
stimulus idx: 4
0
stimulus idx: 5
0
stimulus idx: 6
3
stimulus idx: 7
1
stimulus idx: 8
3
stimulus idx: 9
3
3.0643698573e-01.pt
stimulus idx: 0
5
stimulus idx: 1
4
stimulus idx: 2
4
stimulus idx: 3
4
stimulus idx: 4
4
stimulus idx: 5
4
stimulus idx: 6
5
stimulus idx: 7
5
stimulus idx: 8
5
stimulus idx: 9
5
3.0751633644e-01.pt
stimulus idx: 0
1
stimulus idx: 1
1
stimulus idx: 2
1
stimulus idx: 3
3
stimulus idx: 4
3
stimulus idx: 5
4
stimulus idx: 6
5
stimulus idx: 7
5
stimulus idx: 8
4
stimulus idx: 9
5
3.0905631185e-01.pt
stimulus idx: 0
0
stimulus idx: 1
0
stimulus idx: 2
0
stimulus idx: 3
0
stimulus idx: 4
0
stimulus idx: 5
0
stimulus idx: 6
1
stimulus idx: 7
1
stimulus idx: 8
1
stimulus 

KeyboardInterrupt: 

### calculate the number of eigenvalues with negative real parts of the corresponding response matrix

In [43]:
J = NetConnectivity()
J_mat = ConnectivityMat()

l_baseline = sio.loadmat("data/baselines.mat")['baseline']

l_stimulus = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85]
n_stimulus = len(l_stimulus)

config = 'final_gaussian_classical'
l_selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
l_s_manipulation = ['Inh_part', 'PV_part', 'SST_part', 'VIP_part']
l_title = [s.replace('_', ' ') for s in l_s_manipulation]

n_dimension = 3600

for s_manipulation_idx in range(len(l_s_manipulation)):
    s_manipulation = l_s_manipulation[s_manipulation_idx]
    print(s_manipulation)
    
    if s_manipulation_idx == 0:
        n_neg_eig_val_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus))
    else:
        n_neg_eig_val_rsp_mat = np.zeros((len(l_selected_indices), n_stimulus))

    folder_path = 'models/optimized_files_final_gaussian_classical/'
    model_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f != '.DS_Store']
    sorted_files = sorted(model_names, key=lambda name: float(name.replace('.pt', '')))
    sorted_numbers = sorted([float(f.replace('.pt', '')) for f in sorted_files])

    for model_name_idx in l_selected_indices:

        model_name = sorted_files[model_name_idx]
        print(model_name)

        conn_file_path = folder_path + model_name
        optimized_network = load_optimized_networks_parameters(conn_file_path)

        J_mat_model = load_conn_mat(J, J_mat, optimized_network, n_dimension)  # initialize connectivity and bias

        fr_E = sio.loadmat('simulation_data/fr_E_' + config + '_' + model_name + '.mat')['E']
        fr_P = sio.loadmat('simulation_data/fr_P_' + config + '_' + model_name + '.mat')['P']
        fr_S = sio.loadmat('simulation_data/fr_S_' + config + '_' + model_name + '.mat')['S']
        fr_V = sio.loadmat('simulation_data/fr_V_' + config + '_' + model_name + '.mat')['V']

        tau_values = np.zeros(n_dimension)
        tau_values[:900] = 1/tau_E
        tau_values[900:1800] = 1/tau_P
        tau_values[1800:2700] = 1/tau_S
        tau_values[2700:] = 1/tau_V
        tau_mat = np.diag(tau_values) # gain matrix

        for k in range(n_stimulus):
            if k == 0:
                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(l_baseline[0, 0])
                diag_values[900:1800] = 2 * np.sqrt(l_baseline[0, 1])
                diag_values[1800:2700] = 2 * np.sqrt(l_baseline[0, 2])
                diag_values[2700:] = 2 * np.sqrt(l_baseline[0, 3])
                G = np.diag(diag_values) # gain matrix
            else:
                fr_E_sim = fr_E[:, k]
                fr_P_sim = fr_P[:, k]
                fr_S_sim = fr_S[:, k]
                fr_V_sim = fr_V[:, k]

                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(fr_E_sim)
                diag_values[900:1800] = 2 * np.sqrt(fr_P_sim)
                diag_values[1800:2700] = 2 * np.sqrt(fr_S_sim)
                diag_values[2700:] = 2 * np.sqrt(fr_V_sim)
                G = np.diag(diag_values) # gain matrix

            I = np.eye(n_dimension)
            Jac_mat = np.matmul(tau_mat, (- I + np.matmul(G, J_mat_model))) #  - I + G * J_mat_model # tau_mat * (- I + G * J_mat_model)

            rsp_mat = np.matmul(np.matmul(np.linalg.inv(-Jac_mat), tau_mat), G)
            
            # construct the effective connectivity matrix
            if s_manipulation == 'Inh_part':
                rsp_mat_sub = rsp_mat[900:, 900:]
            elif s_manipulation == 'PV_part':
                rsp_mat_sub = rsp_mat[900:1800, 900:1800]
            elif s_manipulation == 'SST_part':
                rsp_mat_sub = rsp_mat[1800:2700, 1800:2700]
            elif s_manipulation == 'VIP_part':
                rsp_mat_sub = rsp_mat[2700:, 2700:]         
            else:
                pass
            
            evals, evecs = np.linalg.eig(rsp_mat_sub)

            n_neg_eig_val_rsp_mat[model_name_idx, k] = np.sum(np.real(np.array(evals)) < 0)
            
    sio.savemat('data/n_neg_real_eig_val_rsp_' + s_manipulation + '.mat', mdict={'n_neg_real_eig_val': n_neg_eig_val_rsp_mat})

Inh_part
2.9814842343e-01.pt
3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt
PV_part
2.9814842343e-01.pt
3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt
SST_part
2.9814842343e-01.pt
3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt
VIP_part
2.9814842343e-01.pt
3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt


### compute a quantity that reflects the change in mean activity induced by the all-ones vector and by the eigenvector associated with the smallest eigenvalue of the response matrix

In [39]:
J = NetConnectivity()
J_mat = ConnectivityMat()

l_baseline = sio.loadmat("data/baselines.mat")['baseline']

l_stimulus = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85]
n_stimulus = len(l_stimulus)
l_selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

config = 'final_gaussian_classical'

n_dimension = 3600

folder_path = 'models/optimized_files_final_gaussian_classical/'
model_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f != '.DS_Store']
sorted_files = sorted(model_names, key=lambda name: float(name.replace('.pt', '')))
sorted_numbers = sorted([float(f.replace('.pt', '')) for f in sorted_files])
l_projection_coeff_all_ones = np.zeros((4, 10, 10)) * np.nan
l_projected_coeff_patterned = np.zeros((4, 10, 10)) * np.nan

for i in range(4):

    for model_name_idx in l_selected_indices:

        model_name = sorted_files[model_name_idx]
        print(model_name)

        conn_file_path = folder_path + model_name
        optimized_network = load_optimized_networks_parameters(conn_file_path)

        J_mat_model = load_conn_mat(J, J_mat, optimized_network, n_dimension)  # initialize connectivity and bias

        fr_E = sio.loadmat('simulation_data/fr_E_' + config + '_' + model_name + '.mat')['E']
        fr_P = sio.loadmat('simulation_data/fr_P_' + config + '_' + model_name + '.mat')['P']
        fr_S = sio.loadmat('simulation_data/fr_S_' + config + '_' + model_name + '.mat')['S']
        fr_V = sio.loadmat('simulation_data/fr_V_' + config + '_' + model_name + '.mat')['V']

        tau_values = np.zeros(n_dimension)
        tau_values[:900] = 1/tau_E
        tau_values[900:1800] = 1/tau_P
        tau_values[1800:2700] = 1/tau_S
        tau_values[2700:] = 1/tau_V
        tau_mat = np.diag(tau_values) # gain matrix

        for k in range(n_stimulus):
            if k == 0:
                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(l_baseline[0, 0])
                diag_values[900:1800] = 2 * np.sqrt(l_baseline[0, 1])
                diag_values[1800:2700] = 2 * np.sqrt(l_baseline[0, 2])
                diag_values[2700:] = 2 * np.sqrt(l_baseline[0, 3])
                G = np.diag(diag_values) # gain matrix
            else:
                fr_E_sim = fr_E[:, k]
                fr_P_sim = fr_P[:, k]
                fr_S_sim = fr_S[:, k]
                fr_V_sim = fr_V[:, k]

                diag_values = np.zeros(n_dimension)
                diag_values[:900] = 2 * np.sqrt(fr_E_sim)
                diag_values[900:1800] = 2 * np.sqrt(fr_P_sim)
                diag_values[1800:2700] = 2 * np.sqrt(fr_S_sim)
                diag_values[2700:] = 2 * np.sqrt(fr_V_sim)
                G = np.diag(diag_values) # gain matrix

            I = np.eye(n_dimension)
            Jac_mat = np.matmul(tau_mat, (- I + np.matmul(G, J_mat_model))) #  - I + G * J_mat_model # tau_mat * (- I + G * J_mat_model)

            rsp_mat = np.matmul(np.matmul(np.linalg.inv(-Jac_mat), tau_mat), G)

            # construct the effective connectivity matrix
            if i == 0:
                rsp_mat_sub = rsp_mat[900:, 900:]
            elif i == 1:
                rsp_mat_sub = rsp_mat[900:1800, 900:1800]
            elif i == 2:
                rsp_mat_sub = rsp_mat[1800:2700, 1800:2700]
            else:
                rsp_mat_sub = rsp_mat[2700:, 2700:]

            evals, evecs = np.linalg.eig(rsp_mat_sub)
            
            if i == 0:
                v = np.ones(2700)
            else:
                v = np.ones(900)
    
            projection_coeff = np.linalg.inv(evecs) @ v
            scaled_coeff = evals * projection_coeff
            reconstructed_vec = evecs @ scaled_coeff # \delta_r/\gamma
            
            l_projection_coeff_all_ones[i, model_name_idx, k] = np.real(np.mean(reconstructed_vec)) # \delta_r/(\gamma * N)

            # Find index of the leading eigenvalue (largest real part)
            leading_idx = np.argmin(evals.real)

            # Get leading eigenvalue and eigenvector
            leading_eigval = evals[leading_idx]
            leading_eigvec = evecs[:, leading_idx]
            l_projected_coeff_patterned[i, model_name_idx, k] = leading_eigval # lambda_1

sio.savemat('data/projection_coeff_all_ones.mat', mdict={'projection_coeff': l_projection_coeff_all_ones})
sio.savemat('data/projection_coeff_patterned.mat', mdict={'projection_coeff': l_projected_coeff_patterned})

2.9814842343e-01.pt


/var/folders/cw/5d8ywbrn6bsfwdmhjg1nw1840000gn/T/ipykernel_84944/1993432412.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  l_projected_coeff_patterned[i, model_name_idx, k] = leading_eigval


3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt
2.9814842343e-01.pt
3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt
2.9814842343e-01.pt
3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt
2.9814842343e-01.pt
3.0061629415e-01.pt
3.0643698573e-01.pt
3.0751633644e-01.pt
3.0905631185e-01.pt
3.1116414070e-01.pt
3.1124791503e-01.pt
3.1153696775e-01.pt
3.1269198656e-01.pt
3.1403386593e-01.pt
[[[-1.39446233e+01 -2.46109984e+01 -1.81829951e+01 -1.55958793e+01
   -1.47033617e+01 -1.18521812e+01 -1.19671848e+01 -1.18813760e+01
   -1.05539416e+01 -1.10592095e+01]
  [-8.58621668e+00 -8.22295772e+00 -6.76649224e+00